Zuerst die beiden Transkripte Dateien zusammenfügen

In [ ]:
import os
import pandas as pd

In [ ]:
# Google Drive benutzen um Dateien permanent zu speichern

from google.colab import drive
drive.mount('/mnt/drive')

base_path = "/mnt/drive/MyDrive/"

Mounted at /mnt/drive


In [ ]:
# Lese die erste CSV-Datei
df1 = pd.read_csv('/mnt/drive/MyDrive/MasterArbeitDatenLokal/LokaleGruppen/alle_transkripteLokal.csv')

# Lese die zweite CSV-Datei
df2 = pd.read_csv('/mnt/drive/MyDrive/MasterArbeitDaten/alle_transkripteInfluencer.csv')


In [ ]:
# Zusammenführen der DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Speichern des kombinierten DataFrames in eine neue CSV-Datei
combined_df.to_csv('/mnt/drive/MyDrive/AlleTranskripte.csv', index=False)


Die Anzahl an Transkripten pro Gruppe berechnen und dem df hinzufügen

In [ ]:
df = pd.read_csv('/mnt/drive/MyDrive/AlleTranskripte.csv')

# Hinzufügen der Anzahl der Transkripte pro Gruppe zum DataFrame
df['Anzahl_Transkripte'] = df.groupby('Identifier')['Inhalt'].transform('count')

# Speichern des DataFrames in eine CSV-Datei
output_filename = '/mnt/drive/MyDrive/AlleTranskripteMitAnzahl.csv'
df.to_csv(output_filename, index=False)

print(f"Der DataFrame wurde erfolgreich als {output_filename} gespeichert.")

Der DataFrame wurde erfolgreich als /mnt/drive/MyDrive/AlleTranskripteMitAnzahl.csv gespeichert.


# Erstellen eines stratified samples

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('/mnt/drive/MyDrive/AlleTranskripteMitAnzahl.csv')

# Erzeugen einer stratifizierten Stichprobe basierend auf der 'Anzahl_Transkripte' Spalte
train, sample = train_test_split(df, test_size=0.05, stratify=df['Anzahl_Transkripte'], random_state=42)

# Anzeigen der stratifizierten Stichprobe
print(sample)


                       Identifier  \
19227  schwurbelarchiv-g6Sbm9HSmG   
27768  schwurbelarchiv-11I2gdLfJW   
1142   schwurbelarchiv-DIxrNAFZYl   
23473  schwurbelarchiv-QDHQA6B3qC   
1994   schwurbelarchiv-HJmOiP7hRs   
...                           ...   
14459  schwurbelarchiv-lDobCFaR2c   
12836  schwurbelarchiv-k0eohVhfDE   
6577   schwurbelarchiv-lNtnSXDOJk   
24871  schwurbelarchiv-k8vmjIscYR   
15100  schwurbelarchiv-lDobCFaR2c   

                                          Dateiname  \
19227                       IMG_4142_transcript.txt   
27768   video_39@06-05-2020_23-57-51_transcript.txt   
1142       video_2020-12-23_15-54-25_transcript.txt   
23473      video_2019-10-09_14-27-20_transcript.txt   
1994   video_982@10-11-2020_18-18-20_transcript.txt   
...                                             ...   
14459                   hollnadPolit_transcript.txt   
12836  video_123@24-11-2020_15-07-31_transcript.txt   
6577                   Maskenpflicht_transcript.txt   
24871

In [ ]:
# Speichern des DataFrames in eine CSV-Datei
output_filename = '/mnt/drive/MyDrive/AlleTranskripteSample5%.csv'
sample.to_csv(output_filename, index=False)

# Berechnung der Word Error Rate

In [ ]:
import pandas as pd
from nltk.metrics import edit_distance  # Falls nicht installiert, verwende: !pip install nltk

# CSV-Datei laden
df = pd.read_csv("/content/AlleTranskripte5%Versuch.csv", delimiter=";", quotechar='"')  # Ersetze mit dem Pfad zur CSV-Datei

# Spaltennamen anpassen
automated_column = "Inhalt"
manual_column = "Mein Transkript"
annotation_column = "Anmerkung"

# Filterung der Transkripte
df_filtered = df[(df[manual_column].notna()) &
                 (df[manual_column].str.strip() != "") &  # Stellt sicher, dass das Transkript nicht leer ist
                 (~df[annotation_column].str.contains("Übersetzungsfehler", case=False, na=False))].copy()

# WER-Berechnung ohne jiwer
def calculate_wer(reference, hypothesis):
    ref_words = reference.split()
    hyp_words = hypothesis.split()
    distance = edit_distance(ref_words, hyp_words)  # Berechnung des Levenshtein-Abstands auf Wortebene
    return distance / len(ref_words) if len(ref_words) > 0 else float('inf')  # Schutz vor Division durch Null

# WER berechnen für jedes gefilterte Paar
df_filtered['WER'] = df_filtered.apply(lambda row: calculate_wer(row[manual_column], row[automated_column]), axis=1)

# Durchschnittliche WER berechnen
average_wer = df_filtered['WER'].mean()

# Ergebnis anzeigen
print(f"Durchschnittliche Word Error Rate (WER) nach Filterung: {average_wer:.2%}")



Durchschnittliche Word Error Rate (WER) nach Filterung: 37.10%
